## Final Pass


**Notice: This notebook is a modification of [eng.ipynb](https://mlbook.explained.ai/notebooks/index.html) by Terence Parr and Jeremy Howard, which was used by permission of the author.**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from rfpimp_MC import * 

In [ ]:
def evaluate(X, y, n_estimators=50):
    rf = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1, oob_score=True)
    rf.fit(X, y)
    oob = rf.oob_score_
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    print(f"OOB R^2 is {oob:.5f} using {n:,d} tree nodes with {h} median tree depth")
    return rf, oob

In [ ]:
def showimp(rf, X, y):
    features = list(X.columns)
    I = importances(rf, X, y, features=features)
    plot_importances(I, color='#4575b4')

### Recap

Recall that in the last notebook we did the following:
- converted all 'string' features to ordered categorical features;
- label encoded all these features using the value of 0 to represent missing data;
- fixed some remaining problems with `YearMade` and `MachineHoursCurrentMeter`; and 
- replaced missing numeric data by:
    - adding a new boolean (`True` or `False`) feature to say whether or not that value was missing; 
    - replace missing values in the original feature with the median of all values for that feature.

### Next Steps

In our final work on this data we will:
- Break up the `saledate` feature into its components: year, month, day, etc.;
- Ordinal encode `ProductSize`;
- One hot encode `Hydraulics_Flow` and `Enclosure`; and, 
- Take the logarithm of our target variable, `SalePrice`.

### Create a New, More Stable Baseline

To get a more stable baseline, which will be better to compare improvements in our model, we will use 150 decision tress in the random forest instead of the 50 used previously. With more trees, the variance in the OOB $R^2$ is reduced. That means, if we run the model many times the OOB $R^2$ won't change as much (due to randomness of random forest construction). This will give us more confidence that any improvement we see is due to the changes we have made to the data as opposed to just random variations. 

In [ ]:
df_raw = pd.read_feather("bulldozer-train.feather")
df_raw = df_raw.iloc[-100000:]
df = pd.read_feather("bulldozer-train-clean.feather")

In [ ]:
X  = df.drop(['SalePrice','saledate'], axis=1) 
y = df['SalePrice']

rf, oob_all = evaluate(X, y, n_estimators=150)

Notice how the OOB score for our new baseline is slightly higher than what we achieved in the last notebook. This is due to the increased number of trees in the random forest model, since the accuracy tends to increase (up to a point) when more trees are used. 

Let's also recall the feature importances for this model as our next steps are based on exploring what more we can do with the most important features to see if we can improve our OOB score. 

In [ ]:
showimp(rf, X, y)

### Creating New Features from `saledate`

We will now deal with the only `datetime` data type we have in our data: `saledate`. For this procedure we will use Pandas's built-in functionality for datetimes. After extracting the date components, we will convert the original `saledate` feature to an integer, which for most operating systems means the number of seconds since 1 January 1970. 

In [ ]:
def df_split_dates(df,colname):
    df["saleyear"] = df[colname].dt.year
    df["salemonth"] = df[colname].dt.month
    df["saleday"] = df[colname].dt.day
    df["saledayofweek"] = df[colname].dt.dayofweek
    df["saledayofyear"] = df[colname].dt.dayofyear
    df[colname] = df[colname].astype(np.int64)

In [ ]:
df_split_dates(df, 'saledate')

In [ ]:
df_small = df.sample(n=5)
df_small[['saledate', 'saleyear', 'salemonth', 'saleday', 'saledayofweek', 'saledayofyear']]

We don't know beforehand what will be predictive so you should default to adding whatever you can, including things like holidays, or major events, etc. 

And now that we have `saleyear` we can calculate the age of each bulldozer:

In [ ]:
df['age'] = df['saleyear'] - df['YearMade']

Let's train a model and see if these new features have had any impact. 

In [ ]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

rf, oob_dates = evaluate(X, y, n_estimators=150)

This gives us a decent increase in our OOB score. Let's also take a look at the importances, but first, since all the new features we have added are correlated, we will bundle them together as one meta-feature. 

In [ ]:
features = list(df.drop('SalePrice',axis=1).columns)
datefeatures = list(df.filter(regex=("sale*")).columns)

for f in datefeatures:
    features.remove(f)
    
features.remove('YearMade')
features.remove('age')
features += [['YearMade','age']+datefeatures]

I = importances(rf, X, y, features=features)
plot_importances(I.head(15))

### Adjusting the Default Approach

We have said that in practice, or not sure what else to do, we encode categorical features using label encoding, which just assigns an integer to each unique category without considering any order. We want to modify this if we know that the feature is important. In this case, we would use: 
- one hot encoding if the number of unique categories is small, say 10 or less; 
- use ordinal encoding if the feature is ordinal, that is, has a natural ordering. 

We will see examples of both below. 

### Ordinal Encode `ProductSize`

Now let's explore the `ProductSize` feature as it's the second most important according to our model. To see it's connection to our target we will look at the original encoding. 

In [ ]:
temp['ProductSize'].unique()

In [ ]:
temp = df_raw.fillna('nan') 
temp = temp.groupby('ProductSize')['SalePrice'].mean()
temp.head()
temp.sort_values().plot.barh()

The barplot shows that there is a relationship between `ProductSize` and `SalePrice`. This tells us that we should use an ordinal encoding for this feature instead of the label encoding we did before (just assigns an integer to each category value but doesn't respect the ordering). 

In [ ]:
sizes = {None:0, 'Mini':1, 'Compact':1, 'Small':2, 'Medium':3, 'Large / Medium':4, 'Large':5}

df['ProductSize'] = df_raw['ProductSize'].map(sizes).values
df['ProductSize'].unique()

In [ ]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

rf, oob_psize = evaluate(X, y, n_estimators=150)

This gives us another increase, although small. 

### One Hot Encoding

We will skip over the next most important feature, `fiProductClassDesc` (see [Section 8.5](https://mlbook.explained.ai/bulldozer-feateng.html#sec:8.5) for details on that feature), and consider `Enclosure` and `Hydraulics_Flow`.   

We will use one-hot encoding on these features. This technique was outlined in the notebook **Part 1 - Categorical Variables**, so review the appropriate section there if you forget. 

Again, we will need to go back to the original data do this, as the cleaned data has already label encoded these features. Let's first verify that they are not ordinal and don't contain too many unique values. 

In [ ]:
df_raw['Enclosure'].unique()

In [ ]:
df_raw['Hydraulics_Flow'].unique()

It looks like both features fit the criteria for one-hot encoding (although an argument could be made that `Hydraulics_Flow` may be ordinal, given the uncertainty we will assume it is not). Notice also that since we are using the original data we will need to normalize the representations of missing values before we do our encoding.  

#### `Hydraulics_Flow`

To handle this feature we will:
- replace our label encoded version of this feature with the original values;
- normalize the representation of mising values to `np.nan`; and, 
- create a dataframe of the one-hot encoded feature;
- delete the original feature; 
- combine our cleaned data with the one-hot encoded dataframe. 

Note that missing values will be indicated by all `False` (or 0) values when one-hot encoded. 

In [ ]:
df['Hydraulics_Flow'] = df_raw['Hydraulics_Flow'].values
df['Hydraulics_Flow'] = df['Hydraulics_Flow'].replace('None or Unspecified', np.nan)

onehot = pd.get_dummies(df['Hydraulics_Flow'], prefix='Hydraulics_Flow', dtype=bool)
onehot.head(3)

In [ ]:
del df['Hydraulics_Flow']

df = pd.concat([df, onehot], axis=1)

In [ ]:
df.head(3).T

#### `Enclosure`

To handle this feature we will:
- replace our label encoded version of this feature with the original values;
- clean up the data as two of the category values represent the same category;
- normalize the representation of mising values to `np.nan`; and, 
- create a dataframe of the one-hot encoded feature;
- delete the original feature; 
- combine our cleaned data with the one-hot encoded dataframe. 

Note that missing values will be indicated by all `False` (or 0) values when one-hot encoded. 

In [ ]:
df['Enclosure'] = df_raw['Enclosure'].values
df['Enclosure'] = df['Enclosure'].replace('EROPS w AC', 'EROPS AC')
df['Enclosure'] = df['Enclosure'].replace('None or Unspecified', np.nan)

onehot = pd.get_dummies(df['Enclosure'], prefix='Enclosure', dtype=bool)
onehot.head(3)

In [ ]:
del df['Enclosure']
df = pd.concat([df, onehot], axis=1)
df.head(3).T

Let's train a model and see if this has had any impact on performance.

In [ ]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

rf, oob_one_hot = evaluate(X, y, n_estimators=150)

In [ ]:
I = importances(rf, X, y)
plot_importances(I.head(20))

### Using `log(SalePrice)`

The last thing we will try is to train our model on the logarithm of the price instead of just the price. 

In [ ]:
df['SalePrice'].hist()

In [ ]:
np.log(df['SalePrice']).hist()

In [ ]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']
log_y = np.log(y)

rf, oob_log = evaluate(X, log_y, n_estimators=150)

### Summary

So, to recap what we have done, we:
- broke up the `saledate` feature into its components: year, month, day, etc.;
- added an `age` feature; 
- ordinal encoded `ProductSize`;
- one-hot encoded `Hydraulics_Flow` and `Enclosure`; and, 
- used the logarithm of `SalePrice` as our new target variable.

### Results

After all the techniques we have used in this notebook, we have increased our OOB $R^2$ score from the more stable baseline of 0.903 to 0.919, which is a nice increase. This represents an increase in the explained variance of

In [ ]:
print(f"{round(100 * ((1 - 0.903) - (1 - 0.919)) / (1 - 0.903))}%")